In [6]:
from ultralytics import YOLO
import os
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [7]:
# Create new YOLOv8 model
model = YOLO('yolov8n-cls.yaml')


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

In [8]:
# Specify paths of relevant data
SOURCE = 'squares'
TRAIN = os.path.join(SOURCE, "train")
TEST = os.path.join(SOURCE, "val")

In [9]:
# Train model
results = model.train(data=SOURCE, epochs=11, imgsz=64)

Ultralytics YOLOv8.0.210  Python-3.8.16 torch-2.1.1+cpu CPU (11th Gen Intel Core(TM) i5-1145G7 2.60GHz)
engine\trainer: task=classify, mode=train, model=yolov8n-cls.yaml, data=squares, epochs=11, patience=50, batch=16, imgsz=64, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, 

In [10]:
# Make predictions from newly created model
model = YOLO(r'runs\classify\train\weights\best.pt')

a_results = model(os.path.join(TEST, 'a'))
b_results = model(os.path.join(TEST, 'b'))
c_results = model(os.path.join(TEST, 'c'))


image 1/500 c:\Users\Vivian\Documents\Case Study\squares\val\a\0.png: 64x64 a 1.00, b 0.00, c 0.00, 3.1ms
image 2/500 c:\Users\Vivian\Documents\Case Study\squares\val\a\1.png: 64x64 a 0.98, b 0.02, c 0.00, 2.0ms
image 3/500 c:\Users\Vivian\Documents\Case Study\squares\val\a\10.bmp: 64x64 a 0.99, b 0.01, c 0.00, 6.1ms
image 4/500 c:\Users\Vivian\Documents\Case Study\squares\val\a\100.png: 64x64 a 1.00, b 0.00, c 0.00, 5.7ms
image 5/500 c:\Users\Vivian\Documents\Case Study\squares\val\a\101.png: 64x64 a 1.00, b 0.00, c 0.00, 4.0ms
image 6/500 c:\Users\Vivian\Documents\Case Study\squares\val\a\102.bmp: 64x64 a 0.99, b 0.01, c 0.00, 2.0ms
image 7/500 c:\Users\Vivian\Documents\Case Study\squares\val\a\103.jpg: 64x64 a 1.00, b 0.00, c 0.00, 7.0ms
image 8/500 c:\Users\Vivian\Documents\Case Study\squares\val\a\104.jpg: 64x64 a 0.99, b 0.01, c 0.00, 4.0ms
image 9/500 c:\Users\Vivian\Documents\Case Study\squares\val\a\105.bmp: 64x64 a 1.00, b 0.00, c 0.00, 5.0ms
image 10/500 c:\Users\Vivian\Doc

In [11]:
# Parse probability data from each prediction and take the class with the highest probability
def pred_array(results):
    predictions = []
    for result in results:
        probs = result.probs.numpy().data
        pred = probs.argmax()
        predictions.append(pred)
    return predictions

In [12]:
# Create y prediction variable
y_pred = [*pred_array(a_results), *pred_array(b_results), *pred_array(c_results)]

# Create y test variable
y_test = [0]*500 + [1]*500 + [2]*500

In [13]:
# Reformatting y prediction and test variables to reflect actual class names
y_pred = pd.DataFrame(y_pred, columns=['class_id'])
y_pred['class_id'] = y_pred['class_id'].replace([0, 1, 2], ['a', 'b', 'c'])

y_test = pd.DataFrame(y_test, columns=['class_id'])
y_test['class_id'] = y_test['class_id'].replace([0, 1, 2], ['a', 'b', 'c'])

In [14]:
# Calculate accuracy of predictions
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9773333333333334


In [15]:
# Create confusion matrix to visualize prediction results
cm = confusion_matrix(y_test['class_id'], y_pred['class_id'], labels=['a', 'b', 'c'])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['a', 'b', 'c'])
disp.plot()